#### 1. 二叉树节点类

In [86]:
# -*- coding: UTF-8 -*-
class BinNode(object):
    def __init__(self, data, parent=None, lc=None, rc=None, height=0,):
        self.data = data
        self.parent = parent
        self.height = height
        self.lc = lc
        self.rc = rc

    def __eq__(self, other):
        return self.data == other.data

    def __lt__(self, other):
        return self.data < other.data
    
    def insert_aslchild(self,data):
        new_node = BinNode(data,self)
        self.lc = new_node
        return new_node
    
    def insert_asrchild(self,data):
        new_node = BinNode(data,self)
        self.rc = new_node
        return new_node


if __name__ == "__main__":
    n = BinNode(2)
    k = BinNode(2)
    m = n.insert_aslchild(3)
    print m is n.rc
    
    
    

False


#### 2. 二叉树节点工具类
将判断二叉节点状态,性质. 定位与之相关的叔叔,兄弟节点等逻辑封装

In [63]:
class BinNodeUtil(object):
    @staticmethod
    def is_root(node):
        return not node.parent
    @staticmethod
    def is_lchild(node):
        return (not BinNodeUtil.is_root(node))&(node.parent.lc is node)
    @staticmethod
    def is_rchild(node):
        return (not BinNodeUtil.is_root(node))&(node.parent.rc is node)
    @staticmethod
    def has_parent(node):
        return not BinNodeUtil.is_root(node)
    @staticmethod
    def has_lchild(node):
        return not node.lc is None
    @staticmethod
    def has_rchild(node):
        return not node.rc is None
    @staticmethod
    def has_child(node):
        return BinNodeUtil.has_lchild(node) or BinNodeUtil.has_rchild(node)
    @staticmethod
    def has_bothchild(node):
        return BinNodeUtil.has_lchild(node) & BinNodeUtil.has_rchild(node)
    @staticmethod
    def is_leaf(node):
        return not BinNodeUtil.has_child(node)
    @staticmethod
    def sibling(node):
        try:
            return  node.parent.rc if BinNodeUtil.is_lchild(node) else node.parent.lc
        except  AttributeError:
            return None
    @staticmethod
    def uncile(node):
        try:
            parent = node.parent
            return parent.parent.rc if BinNodeUtil.is_lchild(parent) else parent.parent.lc
        except  AttributeError:
            return None

#### 3. 二叉树类